In [1]:
import os

import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
from torchvision import transforms

import pytorch_lightning as pl

In [2]:
transform_list = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor()
    #transforms.Normalize([0.5,],[0.5,])
])

class LogisticRegressionModel(pl.LightningModule):
    def __init__(self, input_dim, output_dim, hidden_dim):
        super(LogisticRegressionModel, self).__init__()
        self.linear1 = torch.nn.Linear(input_dim, hidden_dim)
        self.linear2 = torch.nn.Linear(hidden_dim, output_dim)
        self.relu = torch.nn.ReLU()
     
    def forward(self, x):
        out = self.relu(self.linear1(x.view(x.size(0), -1)))
        out = self.linear2(out)
        return out

    def training_step(self, batch, batch_nb):
        x, y = batch
        loss = F.cross_entropy(self(x), y)
        tensorboard_logs = {'train_loss': loss}
        return {'loss': loss, 'log': tensorboard_logs}

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001, weight_decay=0.01)
    
    def validation_step(self, batch, batch_nb):
        # OPTIONAL
        x, y = batch
        y_hat = self(x)
        return {'val_loss': F.cross_entropy(y_hat, y)}

    def validation_epoch_end(self, outputs):
        # OPTIONAL
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        tensorboard_logs = {'val_loss': avg_loss}
        return {'val_loss': avg_loss, 'log': tensorboard_logs}

    def test_step(self, batch, batch_nb):
        # OPTIONAL
        x, y = batch
        y_hat = self(x)
        return {'test_loss': F.cross_entropy(y_hat, y)}

    def test_epoch_end(self, outputs):
        # OPTIONAL
        avg_loss = torch.stack([x['test_loss'] for x in outputs]).mean()
        logs = {'test_loss': avg_loss}
        return {'test_loss': avg_loss, 'log': logs, 'progress_bar': logs}
    
    def train_dataloader(self):
        # REQUIRED
        return DataLoader(CIFAR10(os.getcwd(), train=True, download=True, transform=transform_list), batch_size=128)

    def val_dataloader(self):
        # OPTIONAL
        return DataLoader(CIFAR10(os.getcwd(), train=True, download=True, transform=transform_list), batch_size=128)

    def test_dataloader(self):
        # OPTIONAL
        return DataLoader(CIFAR10(os.getcwd(), train=False, download=True, transform=transform_list), batch_size=128)

In [ ]:
if torch.cuda.is_available():
    gpus_value=1  # you can continue going on here, like cuda:1 cuda:2....etc. 
    print("Running on the GPU")
else:
    gpus_value=0
    print("Running on the CPU")
    
input_dim = 32*32
hidden_dim = 200
output_dim = 10

model = LogisticRegressionModel(input_dim, output_dim, hidden_dim)
trainer = pl.Trainer(gpus = gpus_value, max_epochs=50)
trainer.fit(model)


GPU available: False, used: False
No environment variable for node rank defined. Set as 0.


Running on the CPU



  | Name    | Type   | Params
-------------------------------
0 | linear1 | Linear | 205 K 
1 | linear2 | Linear | 2 K   
2 | relu    | ReLU   | 0     


Files already downloaded and verified


/home/kunal/anaconda3/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:23: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Files already downloaded and verified


/home/kunal/anaconda3/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:23: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Files already downloaded and verified


/home/kunal/anaconda3/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:23: UserWarning: Did not find hyperparameters at model hparams. Saving checkpoint without hyperparameters.
  warnings.warn(*args, **kwargs)


In [4]:
trainer.test()

Files already downloaded and verified


/home/kunal/anaconda3/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:23: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


--------------------------------------------------------------------------------
TEST RESULTS
{'test_loss': tensor(1.6725)}
--------------------------------------------------------------------------------



In [5]:
import numpy as np
from sklearn.metrics import confusion_matrix

# confusion matrix
cm = np.zeros((10,10), dtype=np.int)

labels_list = []
predicted_list = []

with torch.no_grad():     
    # Iterate through test dataset
    for images, labels in model.test_dataloader():
        images = images.view(-1, input_dim)

        # Forward pass only to get logits/output
        outputs = model(images)

        # Get predictions from the maximum value
        _, predicted = torch.max(outputs.data, 1)
        #print(labels)
        #print(predicted)pre
        
        labels_list.extend(labels.tolist())
        predicted_list.extend(predicted.tolist())
        
cm += confusion_matrix(labels_list, predicted_list)

print(cm)

Files already downloaded and verified
[[339  45  72  30 152  34  74  34 177  43]
 [ 28 505   8  27  26  24  63  28 109 182]
 [ 46  21 254  90 221 115 142  53  33  25]
 [ 26  33  72 247  89 192 184  55  30  72]
 [ 49  19 122  55 378  73 167  68  49  20]
 [ 24  12  81 168  81 390 118  70  19  37]
 [ 15  50  55  67 121  52 543  25  33  39]
 [ 34  24  57  76  85 115  80 433  38  58]
 [ 82  93  27  38  60  28  45  23 546  58]
 [ 28 177  20  49  21  23  53  39 100 490]]


In [6]:
%load_ext tensorboard
%tensorboard --logdir lightning_logs/